## Algoritmos em Grafos: Problema do Fluxo Máximo



## Grafos

> Conjunto de Vértices $V$
> Conjunto de arcos/arestas $E$
> Orientado ou não-orientado
> Custos/Pesos/Distâncias associadas aos arcos/arestas


In [1]:
import math
import random as rand

In [3]:
# Read Instance

def graph_read(file_name):
    global n, m, v_min, v_max, arc_i, arc_j, cap_u, listAdjP, listAdjN, s, t
    
    f = open(file_name,'r+')

    arc_i = {}
    arc_j = {}
    cap_u = {}
    
    infty_cost = 1.0
    v_min = 100000000
    v_max = 0
    a = 0
    row = 0
    for line in f.readlines():
        item_data = line.split()
        if (row >= 4):
            r = row
            a += 1
            i = int(item_data[0])
            arc_i[a] = i
            j = int(item_data[1])
            arc_j[a] = j
            x = float(item_data[2])
            xx = int(x * 10000)
            
            cap_u[a] = xx
            
            ###print a, i,j, item_data[2], x, xx, cap_u[a]
            
            if v_min > min(i,j):
                v_min = min(i,j)
            if v_max < max(i,j):
                v_max = max(i,j)
            #print r, i, j, x
        else:
            if row == 0:
                n = int(item_data[0])
            if row == 1:
                m = int(item_data[0])
            if row == 2:
                s = int(item_data[0])
            if row == 3:
                t = int(item_data[0])
        row += 1
    f.close()

    
    print(" n m s t row v_min v_max ", n, m, s, t, row, v_min, v_max)
    
    # Cria Listas de Adjacencia Positiva e Negativa 
    
    listAdjP = {}
    listAdjN = {}
    for i in range(v_min,v_max+1):
        listAdjP[i] = []
        listAdjN[i] = []

    for l in range(1, m+1):
        listAdjP[arc_i[l]].append(arc_j[l])
        listAdjN[arc_j[l]].append(arc_i[l])


#    print v_min, v_max
#    for i in range(v_min,v_max+1):
#        print 'ListP: ', i, len(listAdjP[i]), listAdjP[i]
#        print 'ListN: ', i, len(listAdjN[i]), listAdjN[i]



In [ ]:
file = "zuse88//elist96d.rmf"
graph_read(file)

 n m s t row v_min v_max  96 528 1 96 532 1 96


In [4]:
def print_graph():
    global n, m, v_min, v_max, arc_i, arc_j, dist, listAdjP, listAdjN, infty_cost, Dijkstra
    print(n,m)
    print(" Lista de Adjacencia Positiva")
    for v in range(1, n+1):
        print(v, len(listAdjP[v]), listAdjP[v])

    print("\n Lista de Adjacencia Negativa")
    for v in range(1, n+1):
        print(v, len(listAdjN[v]), listAdjN[v])

In [6]:
# Generate Instance

def gener_graph_flow():
    global n, m, v_min, v_max, arc_i, arc_j, listAdjP, listAdjN, cap_u, s, t

    arc_i = {}
    arc_j = {}
    cap_u = {}

    v_min = 1
    v_max = 6
    n = 6
    r = 1
    arc_i[r] = 1
    arc_j[r] = 2
    cap_u[r] = 7
    r += 1
    arc_i[r] = 1
    arc_j[r] = 3
    cap_u[r] = 6
    r += 1
    arc_i[r] = 2
    arc_j[r] = 4
    cap_u[r] = 6
    r += 1
    arc_i[r] = 3
    arc_j[r] = 4
    cap_u[r] = 5
    r += 1
    arc_i[r] = 3
    arc_j[r] = 5
    cap_u[r] = 1
    r += 1
    arc_i[r] = 4
    arc_j[r] = 5
    cap_u[r] = 4
    r += 1
    arc_i[r] = 4
    arc_j[r] = 6
    cap_u[r] = 3
    r += 1
    arc_i[r] = 5
    arc_j[r] = 6
    cap_u[r] = 6
    s = 1
    t = 6
    print(" m:", r)
    m = r
    
    # Cria Listas de Adjacencia Positiva e Negativa 
    
    listAdjP = {}
    listAdjN = {}
    for i in range(v_min,v_max+1):
        listAdjP[i] = []
        listAdjN[i] = []

    for l in range(1, m+1):
        listAdjP[arc_i[l]].append(arc_j[l])
        listAdjN[arc_j[l]].append(arc_i[l])

#    print v_min, v_max
#    for i in range(v_min,v_max+1):
#        print 'ListP: ', i, len(listAdjP[i]), listAdjP[i]
#        print 'ListN: ', i, len(listAdjN[i]), listAdjN[i]



In [11]:
gener_graph_flow()

 m: 8


In [7]:

    
        
        
## Find path from s to t in the residual network - Or show there's none.

### Busca em Largura - BFS - 

def bfs_resNet():
    global n, m, v_min, v_max, arc_i, arc_j, dist, listAdjP, listAdjN  
    global ij2arc, s, t, cap_u, pred, f 

    
    pred = {}
    d_visit = {}
    for i in range(v_min, v_max+1):
        d_visit[i] = -1
        pred[i] = -1
    
    q = []
    q.append(s)
    d_visit[s] = 0
    pred[s] = v_max+1
    
    cont1 = 0

    while len(q)>0:
        #print q 
        
        v = q[0]
        q.remove(q[0])
                
        for w in listAdjP[v]:
            cont1 += 1
            #print cont1," | (", v,w,")"
            a = ij2arc[v,w]
            if d_visit[w] == -1 and (cap_u[a] - f[a]) > 0:
                d_visit[w] = d_visit[v] + 1
                pred[w] = v
                if w == t:
                    return True
                q.append(w)
                #print("       v:", w," dist: ", d_visit[w])

        for w in listAdjN[v]:
            cont1 += 1
            #print cont1," | (", w,v,") inv "
            a = ij2arc[w,v]
            if d_visit[w] == -1 and f[a] > 0:
                d_visit[w] = d_visit[v] + 1
                pred[w] = -v
                if w == t:
                    return True
                q.append(w)
                #print("       v:", w," dist: ", d_visit[w])
                
    return False




In [9]:
### Edmonds Karp Alg for Max-Flow

def EK_MaxFlow():
    global n, m, v_min, v_max, arc_i, arc_j, dist, listAdjP, listAdjN  
    global ij2arc, s, t, cap_u, pred, f, print_iters
    
    k = 0
    #gener_graph_flow()
    
    print(" s, t ", s, t)
    ## Flow Var
    f = {}
    path = {}
    
    max_cap = 0
    for a in range(1, m+1):
        f[a] = 0
        if max_cap < cap_u[a]:
            max_cap = cap_u[a]

    ij2arc = {}
    for a in range(1, m+1):
        ij2arc[arc_i[a], arc_j[a]] = a
    
    total_flow = 0
    itera = 0
    while bfs_resNet():
        ## Get path and compute delta
        itera += 1
        #print  " iter pred ", itera, pred 
        k = 0
        v_curr = t
        delta = max_cap
        while pred[v_curr] != v_max + 1:
            v_next = pred[v_curr]
            if v_next > 0:
                a = ij2arc[v_next, v_curr]
                path[k] = a
                if delta > cap_u[a] - f[a]:
                    delta = cap_u[a] - f[a]
            else:
                v_next = -v_next
                a = ij2arc[v_curr, v_next]
                path[k] = -a
                if delta > f[a]:
                    delta = f[a]
                    
            v_curr = v_next                
            k += 1
         
        total_flow += delta
        if print_iters:
            print(" Iteration: ", itera, " Delta:", delta, k, total_flow)
        ## Update Flow
        for qq in range(k):
            a = path[qq]
            if a > 0:
                print(" ",arc_i[a],"->",arc_j[a]," ")
                f[a] += delta
            else:
                print(" ",arc_j[-a],"->",arc_i[-a]," ")
                f[-a] -= delta
                
                
    print(" Total Flow: ", total_flow," Number of Iterations: ", itera," Lenght of last augmenting path: ", k)
    
        
        
    

In [ ]:
## file = "zuse88//elist2560.rmf" Menos caminho aumentante: 9 -- Maior: 45 até iter: 13690 
##         Flow: 4174421
file = "zuse88//elist160d.rmf"
#file = "zuse88//elist500d.rmf"
graph_read(file)

In [12]:
%%time
print_graph()
print_iters = True
EK_MaxFlow()

6 8
 Lista de Adjacencia Positiva
1 2 [2, 3]
2 1 [4]
3 2 [4, 5]
4 2 [5, 6]
5 1 [6]
6 0 []

 Lista de Adjacencia Negativa
1 0 []
2 1 [1]
3 1 [1]
4 2 [2, 3]
5 2 [3, 4]
6 2 [4, 5]
 s, t  1 6
 Iteration:  1  Delta: 3 3 3
  4 -> 6  
  2 -> 4  
  1 -> 2  
 Iteration:  2  Delta: 1 3 4
  5 -> 6  
  3 -> 5  
  1 -> 3  
 Iteration:  3  Delta: 3 4 7
  5 -> 6  
  4 -> 5  
  2 -> 4  
  1 -> 2  
 Iteration:  4  Delta: 1 4 8
  5 -> 6  
  4 -> 5  
  3 -> 4  
  1 -> 3  
 Total Flow:  8  Number of Iterations:  4  Lenght of last augmenting path:  4
CPU times: user 3.73 ms, sys: 1.13 ms, total: 4.87 ms
Wall time: 4.2 ms


In [13]:
#for a in range(1,m+1):
#    if f[a] > 0:
#        print a, arc_i[a], arc_j[a], f[a]

print(" Flows leaving s ")
sum_f = 0
for a in range(1,m+1):
    if f[a] > 0 and arc_i[a] == s:
        print(a, arc_i[a], arc_j[a], f[a]) 
        sum_f += f[a]

print(" Total flow leaving s: ", sum_f)
        
print(" Flows arriving in t ")
sum_f = 0
for a in range(1,m+1):
    if f[a] > 0 and arc_j[a] == t:
        print(a, arc_i[a], arc_j[a], f[a])        
        sum_f += f[a]

print(" Total flow into t: ", sum_f)

        

 Flows leaving s 
1 1 2 6
2 1 3 2
 Total flow leaving s:  8
 Flows arriving in t 
7 4 6 3
8 5 6 5
 Total flow into t:  8


#### Algoritmo -- Pre-Flow Push 

In [14]:
def PR_bfs_init():
    global n, m, v_min, v_max, arc_i, arc_j, dist, listAdjP, listAdjN  
    global ij2arc, s, t, cap_u, pred, f, print_iters, d_visit

    
    d_visit = {}
    for i in range(v_min, v_max+1):
        d_visit[i] = -1
    
    q = []
    q.append(t)
    d_visit[t] = 0
    
    cont1 = 0

    while len(q)>0:
        #print q 
        
        v = q[0]
        q.remove(q[0])
                
        for w in listAdjN[v]:
            cont1 += 1
            #print cont1," | (", w,v,") inv "
            if d_visit[w] == -1:
                d_visit[w] = d_visit[v] + 1
                q.append(w)

    #print " Distances from t in edges"
    #for i in range(v_min, v_max+1):
    #    print i, d_visit[i]


In [15]:
def PR_push_relabel(v):
    global n, m, v_min, v_max, arc_i, arc_j, dist, listAdjP, listAdjN, PR_total_flow, itera 
    global ij2arc, s, t, cap_u, pred, f, print_iters, d_visit, excess, excess_list

    ## Find Admissible Arc
    
    min_d = 2*n+1
    
    for w in listAdjP[v]:
        a = ij2arc[v,w]
        res_cap_a = cap_u[a] - f[a]

        if res_cap_a > 0:
            if d_visit[v] == d_visit[w]+1:
                # (v,w) is admissible
                ## Push min excess(i) and res_cap_a
                if excess[v] < res_cap_a:
                    delta = excess[v]                        
                else:
                    delta = res_cap_a

                f[a] += delta
                excess[v] -=  delta
                if excess[v] == 0 and v != s and v != t:
                    excess_list.remove(v)

                if excess[w] == 0 and w != s and w != t:
                    excess_list.insert(0,w)

                excess[w] += delta

                if w == t:
                    PR_total_flow += delta
                    #print "In PR_Push_Relabel Flow - Itera  ", PR_total_flow, itera
                return True

            # Keeps the minimum height of v neighbors in the residual network
            if min_d > d_visit[w]:
                min_d = d_visit[w]


    for w in listAdjN[v]:
        a = ij2arc[w,v]
        res_cap_a = f[a]

        if res_cap_a > 0:        
            if d_visit[v] == d_visit[w]+1:
                # (w,v) is admissible - (v,w) reverse
                ## Push min excess(i) and res_cap_a

                if excess[v] < res_cap_a:
                    delta = excess[v]
                else:
                    delta = res_cap_a

                f[a] -= delta
                excess[v] -=  delta
                    
                if excess[v] == 0 and v != s and v != t:
                    excess_list.remove(v)
                    
                if excess[w] == 0 and w != s and w != t:
                    excess_list.insert(0,w)
                
                excess[w] += delta
                return True
            
            # Keeps the minimum height of v neighbors in the residual network
            if min_d > d_visit[w]:
                min_d = d_visit[w]

    ## Relabel
    
    d_visit[v] = min_d + 1

    return False


In [16]:
def PR_MaxFlow():
    global n, m, v_min, v_max, arc_i, arc_j, dist, listAdjP, listAdjN, PR_total_flow, itera  
    global ij2arc, s, t, cap_u, pred, f, print_iters, d_visit, excess, excess_list
        
    print(" s, t ", s, t)
    ## Flow Var
    f = {}
    PR_total_flow = 0
    
    ## Pre-Process

    max_cap = 0
    for a in range(1, m+1):
        f[a] = 0
        if max_cap < cap_u[a]:
            max_cap = cap_u[a]

    ij2arc = {}
    for a in range(1, m+1):
        ij2arc[arc_i[a], arc_j[a]] = a
        
    ## Compute Distances in edges from terminal to all other vertices
    
    PR_bfs_init()
    d_visit[s] = n

    ## Init vertices excess
    
    excess = {}
    for v in range(v_min, v_max+1):
        excess[v] = 0

    ## Start flow of arcs leaving s with their maximum capacity
    ## Update excess
    
    excess_list = []
    
    excess[s] = 0
    for w in listAdjP[s]:
        a = ij2arc[s, w]
        f[a] = cap_u[a]
        excess[w] = cap_u[a]
        excess[s] -= cap_u[a]
        excess_list.insert(0, w)
        
    
    ## while there are vertices with excess (except t), i.e.: Active Nodes
    
    itera = 0
    while len(excess_list) > 0:
        itera += 1
        print("\n iter: ", itera)
        print(" List: ", excess_list)
        print(" Dist ", d_visit)
        print(" Excess ", excess)
        print(" Flow ", f)
        print(" cap  ", cap_u)
        
        #pp = input()

        v = excess_list[0]
        PR_push_relabel(v)

    print("\n iter: ", itera)
    print(" List: ", excess_list)
    print(" Dist ", d_visit)
    print(" Excess ", excess)
    print(" Flow ", f)
    print(" cap  ", cap_u)

    ## print " Total Flow: ", total_flow," Number of Iterations: ", itera
    print("In PR_MaxFlow  -  Total Flow:", PR_total_flow," Total Iterations:", itera)



In [17]:
#gener_graph_flow()
print_graph()

6 8
 Lista de Adjacencia Positiva
1 2 [2, 3]
2 1 [4]
3 2 [4, 5]
4 2 [5, 6]
5 1 [6]
6 0 []

 Lista de Adjacencia Negativa
1 0 []
2 1 [1]
3 1 [1]
4 2 [2, 3]
5 2 [3, 4]
6 2 [4, 5]


In [ ]:
%%time
#gener_graph_flow()
file = "zuse88//elist960.rmf"
#file = "zuse88//elist2560.rmf"
graph_read(file)

In [18]:
%%time
PR_MaxFlow()

 s, t  1 6

 iter:  1
 List:  [3, 2]
 Dist  {1: 6, 2: 2, 3: 2, 4: 1, 5: 1, 6: 0}
 Excess  {1: -13, 2: 7, 3: 6, 4: 0, 5: 0, 6: 0}
 Flow  {1: 7, 2: 6, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}
 cap   {1: 7, 2: 6, 3: 6, 4: 5, 5: 1, 6: 4, 7: 3, 8: 6}

 iter:  2
 List:  [4, 3, 2]
 Dist  {1: 6, 2: 2, 3: 2, 4: 1, 5: 1, 6: 0}
 Excess  {1: -13, 2: 7, 3: 1, 4: 5, 5: 0, 6: 0}
 Flow  {1: 7, 2: 6, 3: 0, 4: 5, 5: 0, 6: 0, 7: 0, 8: 0}
 cap   {1: 7, 2: 6, 3: 6, 4: 5, 5: 1, 6: 4, 7: 3, 8: 6}

 iter:  3
 List:  [4, 3, 2]
 Dist  {1: 6, 2: 2, 3: 2, 4: 1, 5: 1, 6: 0}
 Excess  {1: -13, 2: 7, 3: 1, 4: 2, 5: 0, 6: 3}
 Flow  {1: 7, 2: 6, 3: 0, 4: 5, 5: 0, 6: 0, 7: 3, 8: 0}
 cap   {1: 7, 2: 6, 3: 6, 4: 5, 5: 1, 6: 4, 7: 3, 8: 6}

 iter:  4
 List:  [4, 3, 2]
 Dist  {1: 6, 2: 2, 3: 2, 4: 2, 5: 1, 6: 0}
 Excess  {1: -13, 2: 7, 3: 1, 4: 2, 5: 0, 6: 3}
 Flow  {1: 7, 2: 6, 3: 0, 4: 5, 5: 0, 6: 0, 7: 3, 8: 0}
 cap   {1: 7, 2: 6, 3: 6, 4: 5, 5: 1, 6: 4, 7: 3, 8: 6}

 iter:  5
 List:  [5, 3, 2]
 Dist  {1: 6, 2: 2, 3: 2, 4: 2,

In [20]:
#for a in range(1,m+1):
#    if f[a] > 0:
#        print a, arc_i[a], arc_j[a], f[a]

print(" Flows leaving s ")
sum_f = 0
for a in range(1,m+1):
    if f[a] > 0 and arc_i[a] == s:
        print(a, arc_i[a], arc_j[a], f[a]) 
        sum_f += f[a]

print(" Total flow leaving s: ", sum_f)
        
print(" Flows arriving in t ")
sum_f = 0
for a in range(1,m+1):
    if f[a] > 0 and arc_j[a] == t:
        print(a, arc_i[a], arc_j[a], f[a])        
        sum_f += f[a]

print(" Total flow into t: ", sum_f)



 Flows leaving s 
1 1 2 6
2 1 3 2
 Total flow leaving s:  8
 Flows arriving in t 
7 4 6 3
8 5 6 5
 Total flow into t:  8


In [21]:
    for v in range(v_min, v_max+1):
        sum_p = 0
        sum_n = 0
        for w in listAdjP[v]:
            a = ij2arc[v,w]
            sum_p += f[a]
            
        for w in listAdjN[v]:
            a = ij2arc[w,v]
            sum_n += f[a]
        print(" v: ", v, sum_p, sum_n, sum_p - sum_n)


 v:  1 8 0 8
 v:  2 6 6 0
 v:  3 2 2 0
 v:  4 7 7 0
 v:  5 5 5 0
 v:  6 0 8 -8


In [22]:
print(excess)

{1: -8, 2: 0, 3: 0, 4: 0, 5: 0, 6: 8}


In [23]:
print(f)

{1: 6, 2: 2, 3: 6, 4: 1, 5: 1, 6: 4, 7: 3, 8: 5}
